In [1]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [2]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# # filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [3]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [4]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AMCR', 'ATVI', 'AVT', 'BIIB', 'BKR', 'CCI', 'CHD', 'CNC', 'CNXC',
       'ESAB', 'FCFS', 'GEN', 'HZNP', 'IBKR', 'JJC', 'NATI', 'NEX', 'NUVA',
       'PEAK', 'PFG', 'SBNY', 'SCHN', 'SRPT', 'SYNH', 'VC', 'VNT', 'WH',
       'XEL'],
      dtype='object')


In [5]:
df_dif = df_v - df_v.shift(periods=1)
rows, cols = np.where(df_dif == 0)
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
symbols_same_volume = df_v.columns[idx_same_volume]
print(f'symbols with same volume:\n{symbols_same_volume}')

symbols with same volume:
Index(['ABM', 'ACM', 'ACN', 'ADP', 'AGCO', 'ALRM', 'AMCR', 'AMED', 'AMG',
       'AMGN',
       ...
       'VRE', 'VRSK', 'VSAT', 'WERN', 'WEX', 'WSFS', 'WTS', 'WTW', 'XEL',
       'XENE'],
      dtype='object', length=237)


In [6]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['ABNB', 'AIRC', 'ALGM', 'BJ', 'BKI', 'BRBR', 'BTC-USD', 'CARR', 'CDAY',
       'CEG', 'CHK', 'CHX', 'CNXC', 'CR', 'CRWD', 'CTVA', 'DDOG', 'DOW', 'DT',
       'DTM', 'ESAB', 'ETH-USD', 'ETRN', 'FOX', 'FOXA', 'FYBR', 'GEHC', 'GFS',
       'GO', 'GXO', 'HZNP', 'JJC', 'KD', 'LCID', 'MMP', 'MP', 'MRNA', 'NARI',
       'NATI', 'NEX', 'NUVA', 'NVST', 'NVT', 'OGN', 'OTIS', 'PDD', 'PGNY',
       'RIVN', 'SGOV', 'SHC', 'SITM', 'SWAV', 'SYNH', 'VICI', 'VNT', 'VSCO',
       'WH', 'WWE', 'YETI', 'ZM', 'ZS'],
      dtype='object')


In [7]:
symbols_bad_data = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
unique_symbols_bad_data = sorted(list(set(symbols_bad_data)))  # unique symbols
print(f'unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: {len(unique_symbols_bad_data)}')

unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: 299


In [8]:
# get symbols of past model picks
df_picks = pickle_load(path_data_dump, 'df_picks', verbose=verbose)
df_picks.head(1)

,date_end_df_train,max_days_lookbacks,days_lookbacks,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,2023-09-18,120,"[30, 60, 120]",[],[],[],[],[],[],['SHV'],['FTSM'],[],"['CELH', 'DELL', 'LLY']","['CCJ', 'FTI', 'VST']","['ANF', 'ARCH', 'CLS', 'CPRI', 'HEP', 'HRB', '...","['CLMT', 'HZNP', 'SKYW']",[]


In [9]:
# keep only columns with symbols of past picks
df_picks.drop(columns=["date_end_df_train", "max_days_lookbacks", "days_lookbacks"], inplace=True)
df_picks.head(1)

,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,[],[],[],[],[],[],['SHV'],['FTSM'],[],"['CELH', 'DELL', 'LLY']","['CCJ', 'FTI', 'VST']","['ANF', 'ARCH', 'CLS', 'CPRI', 'HEP', 'HRB', '...","['CLMT', 'HZNP', 'SKYW']",[]


In [10]:
df_picks.columns

Index(['sym_freq_15', 'sym_freq_14', 'sym_freq_13', 'sym_freq_12',
       'sym_freq_11', 'sym_freq_10', 'sym_freq_9', 'sym_freq_8', 'sym_freq_7',
       'sym_freq_6', 'sym_freq_5', 'sym_freq_4', 'sym_freq_3', 'sym_freq_2'],
      dtype='object')

In [11]:
import ast

In [12]:
list_of_lists = []

for col in df_picks.columns:
  # convert column values from string to list, e.g. '[]', '[]', '["A", "B", ..]' ... to [], [], ["A", "B", ..], ...
  l_series = df_picks[col].apply(ast.literal_eval)
  # list_of_lists = [l_item for l_item in l_series if l_item]  # this doesn't works  
  for l_item in l_series:
      if l_item:  # 
        list_of_lists.append(l_item)  

symbols_picks = [val for sublist in list_of_lists for val in sublist]
print(f'symbol count from model picks: {len(symbols_picks)}')
# list sorted unique symbols
unique_symbols_picks = sorted(list(set(symbols_picks)))
print(f'unique symbol count from model picks: {len(unique_symbols_picks)}')

symbol count from model picks: 5160
unique symbol count from model picks: 266


In [13]:
def find_common_items(list1, list2):
  """Finds the common items between two lists.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the common items between the two lists.
  """

  common_items = []
  for item in list1:
    if item in list2:
      common_items.append(item)
  return common_items


# code in if block runs only from command line, code will NOT be executed if imported as a module
if __name__ == "__main__": 
  list1 = ["a", "b", "c", "d", "e"]
  list2 = ["b", "c", "d", "f", "g"]
  common_items = find_common_items(list1, list2)
  print(common_items)

['b', 'c', 'd']


In [14]:
# symbols are in past picks but also have bad data 
common_symbols = find_common_items(unique_symbols_picks, unique_symbols_bad_data)
print(common_symbols)

['BKI', 'BTC-USD', 'CHT', 'CMG', 'CVBF', 'CVLT', 'DVA', 'HOLI', 'HY', 'HZNP', 'NATI', 'PKX', 'SFBS', 'SRPT', 'WWE', 'XENE']


In [15]:
def subtract_items(list1, list2):
  """Subtracts items in list2 from items in list1.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the items in list1 that are not in list2.
  """

  subtracted_items = []
  for item in list1:
    if item not in list2:
      subtracted_items.append(item)
  return subtracted_items

In [16]:
symbols_drop = subtract_items(unique_symbols_bad_data, common_symbols)  # don't drop symbols in past picks
symbols_drop .sort()
print(f'len(unique_symbols_bad_data): {len(unique_symbols_bad_data)}')
print(f'len(common_symbols): {len(common_symbols)}')
print(f'len(symbols_drop): {len(symbols_drop)}')

len(unique_symbols_bad_data): 299
len(common_symbols): 16
len(symbols_drop): 283


In [17]:
# symbols_drop = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
# print(f'combined symbols with no volume, same volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
# symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
# symbols_drop .sort()


df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 283


In [18]:
print(f'symbols with no volume:   {len(symbols_no_volume):>5,}')
print(f'symbols with same volume: {len(symbols_same_volume):>5,}')
print(f'symbols with no close:    {len(symbols_no_close):>5,}\n')
print(f'symbols total before drop:                                        {len(df_close.columns):>5,}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop):>5,}\n')
print('                                          symbols     rows')
print(f'df adjOHLCV (df_a) after dropped symbols:   {len(df_a.columns)/5:>5,.0f}    {len(df_a):>5,}')
print(f'df Close (df_c) after dropped symbols:      {len(df_c.columns):>5,}    {len(df_c):>5,}')


symbols with no volume:      28
symbols with same volume:   237
symbols with no close:       61

symbols total before drop:                                        1,567
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c):   283

                                          symbols     rows
df adjOHLCV (df_a) after dropped symbols:   1,284    1,500
df Close (df_c) after dropped symbols:      1,284    1,500


In [19]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean


In [20]:
from myUtils import list_dump

f_symbols_df_close_clean = 'symbols_df_close_clean.csv'  # symbols text file
symbols_df_c = list(df_c)  # column names in df_c
list_dump(symbols_df_c, path_data_dump, f_symbols_df_close_clean)# df_c.columns.to_csv(f_symbols_df_close_clean)

In [21]:
df_a.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-10-13  108.019997  110.610001  106.820000  110.510002  2807600.0   
2023-10-16  111.040001  112.790001  109.639999  111.500000  1675500.0   
2023-10-17  110.099998  112.910004  109.209999  112.400002  1682800.0   
2023-10-18  111.330002  111.330002  108.540001  108.610001  1803500.0   
2023-10-19  109.250000  111.379997  108.860001  109.510002  2028900.0   

                   AA                                               ...  \
                 Open       High        Low      Close      Volume  ...   
Date                                                                ...   
2023-10-13  27.000000  27.250000  26.330000  26.580000   4532500.0  ...   
2023-10-16  27.200001  27.445000  26.783001  26.809999   4977700.0  ...   
2023-10-17  26.299999  28.370001  26.270000  27.770000   6025200.0  ...   
2023-10-18  27.100000  27.430000  26.500000  26.520000   6489400.0  ...   
2023-10-19  26.160000  26.219999  24.215000  24.240000  13562900.0  ...   

                 ZUMZ                                                   ZWS  \
                 Open       High        Low      Close    Volume       Open   
Date                                                                          
2023-10-13  16.480000  16.480000  15.670000  15.810000  409400.0  26.930000   
2023-10-16  15.950000  16.959999  15.950000  16.860001  267000.0  26.580000   
2023-10-17  16.860001  17.760000  16.660000  17.690001  298400.0  26.639999   
2023-10-18  17.520000  17.580000  17.040001  17.150000  358600.0  26.760000   
2023-10-19  17.160000  17.570000  16.780001  17.219999  255200.0  25.920000   

                                                        
                 High        Low      Close     Volume  
Date                                                    
2023-10-13  26.969999  26.299999  26.530001   821600.0  
2023-10-16  26.860001  26.280001  26.820000   944900.0  
2023-10-17  27.139999  26.530001  27.040001   979700.0  
2023-10-18  26.760000  25.850000  25.870001   882300.0  
2023-10-19  26.260000  25.700001  25.780001  1398100.0  

[5 rows x 6420 columns]

In [22]:
df_c.tail()

,A,AA,AAL,AAP,AAPL,AB,ABBV,ABC,ABR,ABT,...,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-13,110.510002,26.580000,11.72,50.750000,178.850006,29.400000,147.960007,188.979996,14.23,90.870003,...,40.560001,104.300003,206.919998,61.799999,41.580002,34.570000,24.150000,174.380005,15.810000,26.530001
2023-10-16,111.500000,26.809999,11.94,53.590000,178.720001,29.670000,147.229996,186.860001,14.60,92.180000,...,40.259998,105.050003,214.970001,62.820000,41.250000,35.919998,24.010000,175.179993,16.860001,26.820000
2023-10-17,112.400002,27.770000,11.94,54.680000,177.149994,29.920000,149.179993,187.449997,14.58,92.139999,...,40.380001,106.589996,212.990005,64.760002,41.099998,36.740002,24.010000,174.259995,17.690001,27.040001
2023-10-18,108.610001,26.520000,11.36,53.240002,175.839996,29.379999,149.279999,189.550003,14.30,95.559998,...,40.209999,106.190002,209.119995,63.020000,39.650002,35.689999,23.650000,169.539993,17.150000,25.870001
2023-10-19,109.510002,24.240000,11.45,51.299999,175.460007,29.309999,145.520004,189.889999,13.94,95.440002,...,39.529999,104.809998,205.990005,63.009998,38.270000,32.240002,23.389999,168.039993,17.219999,25.780001
